<a href="https://colab.research.google.com/github/ramisa7/Team_EEG/blob/main/Milestone1_Data_Science_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Initial Data Analysis



In [85]:
# %pip install google-colab
# from google.colab import drive
# drive.mount('/content/drive')

In [37]:
%pip install pandas
import pandas as pd
import numpy as np

%pip install matplotlib
import matplotlib.pyplot as plt
%pip install seaborn
import seaborn as sns

%pip install pyarrow
%pip install fastparquet

import os

%pip install keras_cv
import keras_cv
import keras
from keras import ops
import tensorflow as tf

import cv2
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import joblib

import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedGroupKFold






Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In the initial data analysis phase, we start by loading the dataset and examining its structure, summary statistics, and distribution of values. This helps us gain insights into the data and understand its characteristics before proceeding with further analysis or modeling. We also visualize the distribution of certain features in the dataset using histograms, bar plots, or box plots. This helps us understand the distribution of values and identify any patterns or outliers in the data.

In [2]:

# train_path = r"train.csv"
# train_data = pd.read_csv(train_path)
# train_data

In [3]:
# print('Train shape:', train_data.shape )

In [4]:
# targets = train_data.columns[-6:]
# print('Targets', list(targets))

In [5]:
# # Summary statistics for numerical columns in train.csv
# print("\nSummary Statistics - Training Data:")
# (train_data.describe())

In [6]:
# #count missing values
# missing_values = train_data.isnull().sum()
# missing_values


In [7]:
# duplicate_rows = train_data[train_data.duplicated()].shape

There is no missing/duplicate data.  

In [8]:
# # Distribution of classes in train.csv
# class_distribution = train_data[targets].sum()

# # convert vote to probability
# train_data[targets] /= train_data[targets].sum(axis=1).values[:, None]
# train_data

In [9]:
# # Visualize distribution of spectrogram_label_offset_seconds in train.csv
# plt.figure(figsize=(10, 6))
# sns.histplot(train_data['spectrogram_label_offset_seconds'], bins=30, kde=True)
# plt.title('Distribution of Spectrogram Label Offset Seconds in Training Data')
# plt.xlabel('Spectrogram Label Offset Seconds')
# plt.ylabel('Count')
# plt.show()

In [10]:
# # Plot the distribution of classes
# plt.figure(figsize=(8, 4))
# sns.barplot(x=class_distribution.index, y=class_distribution.values)
# plt.title('Distribution of Classes in Training Data')
# plt.xlabel('Brain Activity Classes')
# plt.ylabel('Number of Votes')
# plt.show()

In [11]:
# # Relationship between classes and patient_id in train.csv
# plt.figure(figsize=(12, 8))
# sns.boxplot(x='patient_id', y='seizure_vote', data=train_data)
# plt.title('Relationship between Seizure Votes and Patient ID in Training Data')
# plt.xlabel('Patient ID')
# plt.ylabel('Seizure Votes')
# plt.show()

In [ ]:

# import os  # Import the os module for file path manipulation

# # raw data visualization:
# spectrogram_id = 353733	
# subsample_id = 0

# # Construct the file path
# base_dir = "train_spectrograms"  # Specify the base directory
# spect_filename = os.path.join(base_dir, f"{spectrogram_id}.parquet")  


# # try:
# raw_spect_data = pd.read_parquet(spect_filename)
# # Plot each electrode channel
# for column in raw_spect_data.columns:
#     fig, ax = plt.subplots(figsize=(10, 6))  # Create a new figure and axis for each plot
#     ax.plot(raw_spect_data.index, raw_spect_data[column], label=column)
#     ax.set_title("Raw Spectrogram Signal - " + column)
#     ax.set_xlabel("Time")
#     ax.set_ylabel("Voltage")
#     ax.legend()
#     plt.show()


# print("Raw data visualization successful.")
# # except FileNotFoundError:
# #     print(f"File {eeg_filename} not found.")
# # except Exception as e:
# #     print(f"An error occurred: {e}")


In [12]:
# # extractdata from parquet file 

# import pyarrow.parquet as pq
# import os

# # Load the parquet file
# spectrogram_id = 353733	
# base_dir = "train_spectrograms"  
# spect_filename = os.path.join(base_dir, f"{spectrogram_id}.parquet")
# spect_data = pq.read_table(spect_filename).to_pandas()

# # Display the first  row of the data
# print(spect_data.head(5))

# #check the shape 
# print(spect_data.shape)



In [13]:
# spect_data.describe()

# 4. Complex Model (SPECTROGRAM): 

[ WORK ON PROGRESS]

In [14]:
# import pandas as pd
# from scipy.stats import skew, kurtosis

# # Calculate mean, median, variance, skewness, and kurtosis for each channel
# statistical_features = pd.DataFrame({
#     'Channel': spect_data.columns,
#     'Mean': spect_data.mean(),
#     'Median': spect_data.median(),
#     'Variance': spect_data.var(),
#     'Skewness': spect_data.apply(skew),
#     'Kurtosis': spect_data.apply(kurtosis)
# })

# # Display the statistical features
# statistical_features



In [2]:
class CFG:
    verbose = 1  # Verbosity
    seed = 42  # Random seed
    preset = "efficientnetv2_b2_imagenet"  # Name of pretrained classifier
    image_size = [400, 300]  # Input image size
    epochs = 13 # Training epochs
    batch_size = 64  # Batch size
    lr_mode = "cos" # LR scheduler mode from one of "cos", "step", "exp"
    drop_remainder = True  # Drop incomplete batches
    num_classes = 6 # Number of classes in the dataset
    fold = 0 # Which fold to set as validation data
    class_names = ['Seizure', 'LPD', 'GPD', 'LRDA','GRDA', 'Other']
    label2name = dict(enumerate(class_names))
    name2label = {v:k for k, v in label2name.items()}

In [16]:
#Sets value for random seed to produce similar result in each run.

keras.utils.set_random_seed(CFG.seed)

In [17]:
# # Meta Data: 

# # Train + Valid
# df = pd.read_csv('train.csv')
# df['eeg_path'] = f'train_eegs/'+df['eeg_id'].astype(str)+'.parquet'
# df['spect_path'] = f'train_spectrograms/'+df['spectrogram_id'].astype(str)+'.parquet'
# df['class_name'] = df.expert_consensus.copy()
# df['class_label'] = df.expert_consensus.map(CFG.name2label)
# # display(df.tail(5))
# # display(df.head(5))




# # Test
# test_df =  pd.read_csv('test.csv')
# test_df['eeg_path'] = f'test_eegs/'+test_df['eeg_id'].astype(str)+'.parquet'
# test_df['spec_path'] = f'test_spectrograms/'+test_df['spectrogram_id'].astype(str)+'.parquet'
# # display(test_df)

### TRAIN, TEST AND VALID SPLIT

In [26]:
# Load the data

#df is for train and valid both
df = pd.read_csv('train.csv')
df['eeg_path'] = f'train_eegs/'+df['eeg_id'].astype(str)+'.parquet'
df['spect_path'] = f'train_spectrograms/'+df['spectrogram_id'].astype(str)+'.parquet'
df['class_name'] = df.expert_consensus.copy()
df['class_label'] = df.expert_consensus.map(CFG.name2label)

# Split the data into a training set and a test set using an 90-10 split
df, test_df = train_test_split(df, test_size=0.1, random_state=CFG.seed, stratify=df['class_label'])



# Give me the shape of the training, validation, and test sets
print(f"Training set shape: {df.shape}")
print(f"Test set shape: {test_df.shape}")


Training set shape: (96120, 19)
Test set shape: (10680, 19)


In [27]:
# turn parquet files into numpy arrays for faster loading 
# Define a function to process a single spectrogram_id
def process_spec(spectrogram_id, split="train_"):
    spect_path = f"{split}spectrograms/{spectrogram_id}.parquet"
    spect = pd.read_parquet(spect_path)
    spect = spect.fillna(0).values[:, 1:].T
    spect = spect.astype("float32")
    np.save(f"{split}spectrograms/{spectrogram_id}.npy", spect)

# Get unique spec_ids of train+valid data
spectrogram_ids = df["spectrogram_id"].unique()

# Parallelize the processing using joblib for training+validating data
_ = joblib.Parallel(n_jobs=-1, backend="loky")(
    joblib.delayed(process_spec)(spec_id, "train_")
    for spec_id in tqdm(spectrogram_ids, total=len(spectrogram_ids))
)

# Get unique spec_ids of test data
test_spec_ids = test_df["spectrogram_id"].unique()

# Parallelize the processing using joblib for test data
_ = joblib.Parallel(n_jobs=-1, backend="loky")(
    joblib.delayed(process_spec)(spec_id, "train_")
    for spec_id in tqdm(test_spec_ids, total=len(test_spec_ids))
)

  0%|          | 0/10878 [00:00<?, ?it/s]

  0%|          | 0/4698 [00:00<?, ?it/s]

This below block of code is used to process spectrogram data in parallel for both training and testing datasets. It first retrieves the unique spectrogram IDs from the training data. Then, it uses the joblib library to parallelize the processing of these spectrograms. The 'process_spec' function is applied to each spectrogram ID. This function is called with joblib.delayed, which means the function calls are executed in parallel, not immediately. A progress bar is created for this operation using tqdm, with the total number of iterations set to the number of unique spectrogram IDs. The results of the parallel computation are not stored, indicating that the 'process_spec' function likely performs an operation such as saving the processed spectrograms to disk. The same steps are then repeated for the test data.

In [30]:
print(len(spectrogram_ids), len(test_spec_ids))

10878 4698


In [29]:
# print(spectrogram_ids)
#print (test_spec_ids)


In [ ]:
# for spectrogram_id in spectrogram_ids:
#     process_spec(spectrogram_id, split="train_")
     
#for spectrogram_id in test_spec_ids:
#     process_spec(spectrogram_id, split="train_")

In [18]:
# #load the npy files
# spect = np.load('train_spectrograms/353733.npy')
# print(spect.shape)

# # show the first 5 rows of the data
# print(spect[:10])



In [31]:
#train+valid and test in npy
df['spect2_path'] = f'train_spectrograms/'+ df['spectrogram_id'].astype(str)+'.npy'
test_df['spect2_path'] = f'train_spectrograms/'+test_df['spectrogram_id'].astype(str)+'.npy'
display(test_df)
display(df)

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,eeg_path,spect_path,class_name,class_label,spect2_path
69187,2259539799,153,694.0,1391458063,153,694.0,531396712,2641,GPD,5,0,10,0,0,0,train_eegs/2259539799.parquet,train_spectrograms/1391458063.parquet,GPD,2,train_spectrograms/1391458063.npy
40812,1516493459,1,2.0,785385791,1,2.0,2620430346,62197,LRDA,0,0,0,3,0,0,train_eegs/1516493459.parquet,train_spectrograms/785385791.parquet,LRDA,3,train_spectrograms/785385791.npy
33867,2009915165,18,90.0,668167696,18,90.0,1080254207,53636,Seizure,3,0,0,0,0,0,train_eegs/2009915165.parquet,train_spectrograms/668167696.parquet,Seizure,0,train_spectrograms/668167696.npy
5039,810249434,0,0.0,82847957,0,0.0,265802662,44246,Other,0,0,0,0,0,1,train_eegs/810249434.parquet,train_spectrograms/82847957.parquet,Other,5,train_spectrograms/82847957.npy
74926,3338940223,2,4.0,1498477269,2,4.0,2762737698,34153,GPD,0,0,7,0,3,3,train_eegs/3338940223.parquet,train_spectrograms/1498477269.parquet,GPD,2,train_spectrograms/1498477269.npy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16244,3941153433,5,100.0,322284022,5,100.0,3165798648,4814,Seizure,3,0,0,0,0,0,train_eegs/3941153433.parquet,train_spectrograms/322284022.parquet,Seizure,0,train_spectrograms/322284022.npy
2454,1470287332,11,48.0,37007296,11,48.0,708989365,2338,GPD,0,0,3,0,0,0,train_eegs/1470287332.parquet,train_spectrograms/37007296.parquet,GPD,2,train_spectrograms/37007296.npy
79821,2789122958,4,30.0,1616768321,4,30.0,4249349688,38387,Other,0,0,4,0,2,6,train_eegs/2789122958.parquet,train_spectrograms/1616768321.parquet,Other,5,train_spectrograms/1616768321.npy
40158,3832216758,1,2.0,766908178,1,2.0,1801826023,2573,Other,0,0,0,0,0,3,train_eegs/3832216758.parquet,train_spectrograms/766908178.parquet,Other,5,train_spectrograms/766908178.npy


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,eeg_path,spect_path,class_name,class_label,spect2_path
77135,1744516630,2,6.0,1554060622,2,6.0,3671005733,5393,Seizure,3,0,0,0,0,0,train_eegs/1744516630.parquet,train_spectrograms/1554060622.parquet,Seizure,0,train_spectrograms/1554060622.npy
89814,3288991214,0,0.0,1836830050,0,0.0,2340601286,65378,Other,0,0,0,0,0,1,train_eegs/3288991214.parquet,train_spectrograms/1836830050.parquet,Other,5,train_spectrograms/1836830050.npy
37810,662625553,7,68.0,744859685,7,68.0,2026803436,29192,Seizure,3,1,0,1,0,0,train_eegs/662625553.parquet,train_spectrograms/744859685.parquet,Seizure,0,train_spectrograms/744859685.npy
103265,1480985066,338,1450.0,2063104016,338,1450.0,2691322208,8033,GRDA,0,0,0,0,3,0,train_eegs/1480985066.parquet,train_spectrograms/2063104016.parquet,GRDA,4,train_spectrograms/2063104016.npy
3422,2080477284,3,14.0,55011312,3,14.0,3215853517,50648,GRDA,0,0,0,0,3,0,train_eegs/2080477284.parquet,train_spectrograms/55011312.parquet,GRDA,4,train_spectrograms/55011312.npy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89380,577794603,17,50.0,1834251506,17,50.0,3168356345,30631,Other,1,0,0,1,0,2,train_eegs/577794603.parquet,train_spectrograms/1834251506.parquet,Other,5,train_spectrograms/1834251506.npy
19744,2147495892,4,18.0,386966986,4,18.0,2752832790,37447,Seizure,5,0,0,0,0,0,train_eegs/2147495892.parquet,train_spectrograms/386966986.parquet,Seizure,0,train_spectrograms/386966986.npy
7165,1748851242,11,142.0,134385207,11,142.0,3389451794,61174,LPD,0,3,0,0,0,0,train_eegs/1748851242.parquet,train_spectrograms/134385207.parquet,LPD,1,train_spectrograms/134385207.npy
80528,985265752,6,20.0,1632325764,6,20.0,3345309326,2427,LRDA,0,0,0,3,0,0,train_eegs/985265752.parquet,train_spectrograms/1632325764.parquet,LRDA,3,train_spectrograms/1632325764.npy


This DataLoader first reads npy spectrogram files and extracts labeled subsamples using specified offset values. Then, it converts the spectrogram data into log spectrogram and applies the popular signal augmentation MixUp.

Note that, we are converting the mono channel signal to a 3-channel signal for using "ImageNet" weights of pretrained model.

In [38]:
def build_augmenter(dim=CFG.image_size):
    augmenters = [
        keras_cv.layers.MixUp(alpha=2.0),
        keras_cv.layers.RandomCutout(height_factor=(1.0, 1.0),
                                     width_factor=(0.06, 0.1)), # freq-masking
        keras_cv.layers.RandomCutout(height_factor=(0.06, 0.1),
                                     width_factor=(1.0, 1.0)), # time-masking
    ]
    
    def augment(img, label):
        data = {"images":img, "labels":label}
        for augmenter in augmenters:
            if tf.random.uniform([]) < 0.5:
                data = augmenter(data, training=True)
        return data["images"], data["labels"]
    
    return augment

    



In [39]:
def build_decoder(with_labels=True, target_size=CFG.image_size, dtype=32):
    def decode_signal(path, offset=None):
        # Read .npy files and process the signal
        file_bytes = tf.io.read_file(path)
        sig = tf.io.decode_raw(file_bytes, tf.float32)
        sig = sig[1024//dtype:]  # Remove header tag
        sig = tf.reshape(sig, [400, -1])
        
        # Extract labeled subsample from full spectrogram using "offset"
        if offset is not None: 
            offset = offset // 2  # Only odd values are given
            sig = sig[:, offset:offset+300]
            
            # Pad spectrogram to ensure the same input shape of [400, 300]
            pad_size = tf.math.maximum(0, 300 - tf.shape(sig)[1])
            sig = tf.pad(sig, [[0, 0], [0, pad_size]])
            sig = tf.reshape(sig, [400, 300])
        
        # Log spectrogram 
        sig = tf.clip_by_value(sig, tf.math.exp(-4.0), tf.math.exp(8.0)) # avoid 0 in log
        sig = tf.math.log(sig)
        
        # Normalize spectrogram
        sig -= tf.math.reduce_mean(sig)
        sig /= tf.math.reduce_std(sig) + 1e-6
        
        # Mono channel to 3 channels to use "ImageNet" weights
        sig = tf.tile(sig[..., None], [1, 1, 3])
        return sig
    
    def decode_label(label):
        label = tf.one_hot(label, CFG.num_classes)
        label = tf.cast(label, tf.float32)
        label = tf.reshape(label, [CFG.num_classes])
        return label
    
    def decode_with_labels(path, offset=None, label=None):
        sig = decode_signal(path, offset)
        label = decode_label(label)
        return (sig, label)
    
    return decode_with_labels if with_labels else decode_signal


def build_dataset(paths, offsets=None, labels=None, batch_size=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=False, repeat=True, shuffle=1024, 
                  cache_dir="", drop_remainder=False):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter()
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = (paths, offsets) if labels is None else (paths, offsets, labels)
    
    try:
        ds = tf.data.Dataset.from_tensor_slices(slices)
        ds = ds.map(decode_fn, num_parallel_calls=AUTO)
        ds = ds.cache(cache_dir) if cache else ds
        ds = ds.repeat() if repeat else ds
        if shuffle: 
            ds = ds.shuffle(shuffle, seed=CFG.seed)
            opt = tf.data.Options()
            opt.experimental_deterministic = False
            ds = ds.with_options(opt)
        ds = ds.batch(batch_size, drop_remainder=drop_remainder)
        ds = ds.map(augment_fn, num_parallel_calls=AUTO) if augment else ds
        ds = ds.prefetch(AUTO)
        return ds
    except Exception as e:
        print("Error building dataset:", e)
        return None


In the following code snippet, the data is divided into 5 folds. Note that, the groups argument is used to prevent any overlap of patients between the training and validation sets, thus avoiding potential data leakage issues. Additionally, each split is stratified based on the class_label, ensuring a uniform distribution of class labels in each fold.

In [40]:
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=CFG.seed)

df["fold"] = -1
df.reset_index(drop=True, inplace=True)
for fold, (train_idx, valid_idx) in enumerate(
    sgkf.split(df, y=df["class_label"], groups=df["patient_id"])
):
    df.loc[valid_idx, "fold"] = fold
df.groupby(["fold", "class_name"])[["spectrogram_id"]].count().T

fold               0                                     1                    \
class_name       GPD  GRDA   LPD  LRDA Other Seizure   GPD  GRDA   LPD  LRDA   
spectrogram_id  3899  2765  2864  2515  3653    3759  2200  3854  3599  1853   

fold            ...     3                         4                          \
class_name      ...   LPD  LRDA Other Seizure   GPD  GRDA   LPD  LRDA Other   
spectrogram_id  ...  1935  3039  3714    3719  3358  4346  2649  3163  3244   

fold                    
class_name     Seizure  
spectrogram_id    4024  

[1 rows x 30 columns]

In [41]:
# Sample from full train+valid data
sample_df = df.groupby("spectrogram_id").head(1).reset_index(drop=True)
train_df = sample_df[sample_df.fold != CFG.fold]
valid_df = sample_df[sample_df.fold == CFG.fold]
print(f"# Num Train: {len(train_df)} | Num Valid: {len(valid_df)}")

# Num Train: 8503 | Num Valid: 2375


In [42]:
# train_df

Only first sample for each spectrogram_id is used in order to keep the dataset size managable.

In [43]:
import keras_cv

# Train
train_paths = train_df.spect2_path.values
train_offsets = train_df.spectrogram_label_offset_seconds.values.astype(int)
train_labels = train_df.class_label.values
train_ds = build_dataset(train_paths, train_offsets, train_labels, batch_size=CFG.batch_size,
                         repeat=True, shuffle=True, augment=True, cache=True)


In [44]:
# print(train_ds)

In [45]:
# Valid
valid_paths = valid_df.spect2_path.values
valid_offsets = valid_df.spectrogram_label_offset_seconds.values.astype(int)
valid_labels = valid_df.class_label.values
valid_ds = build_dataset(valid_paths, valid_offsets, valid_labels, batch_size=CFG.batch_size,
                         repeat=False, shuffle=False, augment=False, cache=True)

In [51]:
# print(valid_ds)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 400, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 6), dtype=tf.float32, name=None))>


In [49]:
# Test
test_paths = test_df.spect2_path.values
test_offsets = test_df.spectrogram_label_offset_seconds.values.astype(int)
test_labels = test_df.class_label.values
test_ds = build_dataset(test_paths, test_offsets, test_labels, batch_size=CFG.batch_size,
                         repeat=False, shuffle=False, augment=False, cache=True)

In [50]:
# print(test_ds)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 400, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 6), dtype=tf.float32, name=None))>


In [52]:
LOSS = keras.losses.KLDivergence()

In [53]:
# Build Classifier
model = keras_cv.models.ImageClassifier.from_preset(
    CFG.preset, num_classes=CFG.num_classes
)

# Compile the model  
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              loss=LOSS)

# Model Sumamry
model.summary()

Model: "image_classifier_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficient_net_v2b2_backbone     │ (None, None, None,     │     8,769,374 │
│ (EfficientNetV2Backbone)        │ 1408)                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ avg_pool                        │ (None, 1408)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 6)              │         8,454 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,777,828 (33.48 MB)

 Trainable params: 8,695,540 (33.17 MB)

 Non-trainable params: 82,288 (321.44 KB)

A well-structured learning rate schedule is essential for efficient model training, ensuring optimal convergence and avoiding issues such as overshooting or stagnation.

In [54]:
import math

def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 5e-5, 6e-6 * batch_size, 1e-5
    lr_ramp_ep, lr_sus_ep, lr_decay = 3, 0, 0.75

    def lrfn(epoch):  # Learning rate update function
        if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
        elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        elif mode == 'step': lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # Create lr callback
lr_cb = get_lr_callback(CFG.batch_size, mode=CFG.lr_mode, plot=True)



In [56]:
ckpt_cb = keras.callbacks.ModelCheckpoint("best_model.keras",
                                         monitor='val_loss',
                                         save_best_only=True,
                                         save_weights_only=False,
                                         mode='min')

In [57]:
# Train the model
history = model.fit(train_ds,
                    validation_data=valid_ds,
                    epochs=CFG.epochs,
                    callbacks=[lr_cb, ckpt_cb],
                    verbose=CFG.verbose)



Epoch 1/13
     10/Unknown 366s 33s/step - loss: 1.5142

KeyboardInterrupt: 

In [ ]:
# Load the best model
model = keras.models.load_model("best_model.keras", compile=False)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              loss=LOSS)

# Evaluate the model
loss, acc = model.evaluate(test_ds, verbose=CFG.verbose)
print(f"Test Accuracy: {acc:.4f}")